In [155]:
import sqlite3 as sql
import csv
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import lxml


In [156]:
def csv_parser(csv_reader, header: str):
    _header = next(csv_reader)
    headerIndex = _header.index(header)

    # now create an empty list to append the addresses to
    data_list = []

    # loop through CSV file and append to address_list
    for line in csv_reader:
        all_data = line[headerIndex]
        data_list.append(all_data)
    return data_list

In [157]:

states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

In [158]:
# TODO: Take all the data at each key[value][index] position and create a dict for each
# index postion, then append that dict to an array of dicts


In [159]:

# Open CSV file to append data to:
# with open('new_meeting_data.csv', 'w') as f:
#     csv_writer = csv.DictWriter(f, field_names=list(data.keys()))

# program start:
# start = time.time()

# end of program timer
# end = time.time()
# t = end - start
# print(f"Program took: {t} seconds")


In [160]:
csv_filename = './meetings.csv'

# create func to get list of links:
def get_links(csv_filename):
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        link_list = csv_parser(csv_reader, 'link')
    return link_list


In [161]:
link_list = get_links(csv_filename)

In [162]:
# The func can only take one link at a time so that each row gets 
# created in the correct order.
# THIS WAS INSERTED INTO meeting_data_scraper
def meeting_row_parser(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {}
    row['name'] = item0['name']
    row['address'] = item0['address']
    row['city'] = item0['city']
    row['state'] = item0['state']

    # now add item1 to the row data
    row['day'] = '|'.join(item1['day'])
    row['time'] = '|'.join(item1['time'])
    row['info'] = '|'.join(item1['info'])

    # now return the row data dictionary
    return row


    # empty list to append individual data to
    dict_list = []

    # Now loop through the CSV file and append to the data list:
    for k, v in data.items():
        row_data = {k: v}
        dict_list.append(row_data)
    return dict_list

def meeting_data_scraper(link):
    
    def get_address_data(soup):
        
        try:
            address_tag = soup.address
            address = address_tag.contents[1]
            
            meeting_name = soup.find(
                'div', class_='fui-card-body').find(class_='weight-300')
            name = meeting_name.contents[1]
            
            city_tag = meeting_name.find_next('a')
            city = city_tag.contents[0]
            
            state_tag = city_tag.find_next('a')
            state = state_tag.contents[0]
            return {'name': name, 'address': address, 'city': city, 'state': state}
            
        except IndexError as ie:
            print(f"Index Error: {ie}")
            try:
                return {'name': name, 'address': address, 'city': city, 'state': 'state'}
            except UnboundLocalError as ule:
                print(f"UnboundLocalError: {ule}")
            try:
                return {'name': name, 'address': address, 'city': 'city', 'state': state}
            except UnboundLocalError as ule:
                print(f"UnboundLocalError: {ule}")
            try:
                return {'name': name, 'address': 'address', 'city': city, 'state': state}
            except UnboundLocalError as ule:
                print(f"UnboundLocalError: {ule}")
            try:
                return {'name': 'name', 'address': address, 'city': city, 'state': state}
            except UnboundLocalError as ule:
                print(f"UnboundLocalError: {ule}")
            

        
    def get_table_data(soup):
        try:
            info_table = soup.find('table', class_='table fui-table')
            # obtain all the columns from <th>
            headers = []
            for i in info_table.find_all('th'):
                title = i.text
                headers.append(title.lower())

                # now create a dataframe:
            df = pd.DataFrame(columns=headers)

        
            # Now create the foor loop to fill dataframe
            # a row is under the <tr> tags and data under the <td> tags
            for j in info_table.find_all('tr')[1:]:
                # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
                #     print("No info table found")
                row_data = j.find_all('td')
                row = [i.text for i in row_data]
                length = len(df)
                df.loc[length] = row

            # data['day'].append(df['day'].to_list())
            # data['time'].append(df['time'].to_list())
            # data['info'].append(df['info'].to_list())
            day = df['day'].to_list()
            time = df['time'].to_list()
            info = df['info'].to_list()

            # now return data
            return {'day': day, 'time': time, 'info': info}
        
        except AttributeError as ae:
            print(f"info_table.find_all('tr')[1:] raised error: {ae}")
            return {'day': 'day', 'time': 'time', 'info': 'info'}


    def meeting_row_parser(item0, item1):
        """
        :param item0: This is the address data in a dictionary. Use the following keys to access
        the data -> Keys: 'name' - 'address' - 'city' - 'state' 
        :param item1: This is the meeting details data in a dictionary. Use the following keys to
        access the data -> Keys: 'day' - 'time' - 'info'
        
        create a final dictionary that will be used to store the information in the database as one row. 
        I will need to join the list items to create one string with a | seperating each item so I can 
        split the string when retrieving the data.
        """
        row = {}
        try:
            row['name'] = item0['name']
            row['address'] = item0['address']
            row['city'] = item0['city']
            row['state'] = item0['state']
        except Exception as e:
            print(e)
            row['name'] = 'name'
            row['address'] = 'address'
            row['city'] = 'city'
            row['state'] = 'state'

        # now add item1 to the row data
        row['day'] = ' | '.join(item1['day'])
        row['time'] = ' | '.join(item1['time'])
        row['info'] = ' | '.join(item1['info'])

        # now return the row data dictionary
        return row
    
    # STEP 1: Get the webpage response obj from requests
    page = requests.get(link)
    
    # STEP 2: Get Soup object for html parsing
    soup = bs(page.text, "lxml")

    # Create two dicts with the following keys
    address_dict = get_address_data(soup)
    details_dict = get_table_data(soup)

    d = [address_dict, details_dict]
    row_data = meeting_row_parser(d[0], d[1])
    
    return row_data


In [163]:
# Now use the scraper function to scrape each link in that list
# and store the data in the d variable for dict
row_data = []
for link in link_list[1:]:
    row = meeting_data_scraper(link)
    # data = add_row_data_to_list(row)
    row_data.append(row)


In [131]:
print(len(row_data))

520


In [149]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import Pool
executor = ThreadPoolExecutor(max_workers=2)


def create_soup_obj_from_page(pages):
    soup_obj = []
    for page in pages:
        soup = bs(page.text, 'lxml')
        soup_obj.append(soup)
    return soup_obj



In [153]:
# fetch pages from each link:
def fetch_page_data(link):
    page = requests.get(link)
    soup = bs(page.text, 'lxml')
    return soup

link_list = get_links(csv_filename)

In [154]:
soup_obj = Pool.map(fetch_page_data, link_list[1])


TypeError: map() missing 1 required positional argument: 'iterable'

In [ ]:
# TODO: CREATE FUNCTION THAT WILL GET THE PAGE CONTENTS AND CREATE SOUP OBJ
def create_soup_obj_from_contents(link):
    page = requests.get(link)
    soup = bs(page.text, 'lxml')
    return soup

# TODO: CREATE FUNCTION THAT WILL RETRIEVE THE ADDRESS DATA AND RETURN DICT
def get_address_data(soup):

    try:
        address_tag = soup.address
        address = address_tag.contents[1]

        meeting_name = soup.find(
            'div', class_='fui-card-body').find(class_='weight-300')
        name = meeting_name.contents[1]

        city_tag = meeting_name.find_next('a')
        city = city_tag.contents[0]

        state_tag = city_tag.find_next('a')
        state = state_tag.contents[0]
        return {'name': name, 'address': address, 'city': city, 'state': state}

    except IndexError as ie:
        print(f"Index Error: {ie}")
        try:
            return {'name': name, 'address': address, 'city': city, 'state': 'state'}
        except UnboundLocalError as ule:
            print(f"UnboundLocalError: {ule}")
        try:
            return {'name': name, 'address': address, 'city': 'city', 'state': state}
        except UnboundLocalError as ule:
            print(f"UnboundLocalError: {ule}")
        try:
            return {'name': name, 'address': 'address', 'city': city, 'state': state}
        except UnboundLocalError as ule:
            print(f"UnboundLocalError: {ule}")
        try:
            return {'name': 'name', 'address': address, 'city': city, 'state': state}
        except UnboundLocalError as ule:
            print(f"UnboundLocalError: {ule}")

# TODO: CREATE FUNCTION THAT WILL RETRIEVE THE DETAILS DATA FROM AN HTML TABLE
def get_table_data(soup):
    try:
        info_table = soup.find('table', class_='table fui-table')
        # obtain all the columns from <th>
        headers = []
        for i in info_table.find_all('th'):
            title = i.text
            headers.append(title.lower())

            # now create a dataframe:
        df = pd.DataFrame(columns=headers)

        # Now create the foor loop to fill dataframe
        # a row is under the <tr> tags and data under the <td> tags
        for j in info_table.find_all('tr')[1:]:
            # if info_table.find_all('tr')[1:] == AttributeError.NoneType:
            #     print("No info table found")
            row_data = j.find_all('td')
            row = [i.text for i in row_data]
            length = len(df)
            df.loc[length] = row

        # data['day'].append(df['day'].to_list())
        # data['time'].append(df['time'].to_list())
        # data['info'].append(df['info'].to_list())
        day = df['day'].to_list()
        time = df['time'].to_list()
        info = df['info'].to_list()

        # now return data
        return {'day': day, 'time': time, 'info': info}

    except AttributeError as ae:
        print(f"info_table.find_all('tr')[1:] raised error: {ae}")
        return {'day': 'day', 'time': 'time', 'info': 'info'}

# TODO: PASS THOSE TWO DICTS TO A FUNCTION THAT WILL PARSE AND COMBINE THEM:
def meeting_row_parser(item0, item1):
    """
    :param item0: This is the address data in a dictionary. Use the following keys to access
    the data -> Keys: 'name' - 'address' - 'city' - 'state' 
    :param item1: This is the meeting details data in a dictionary. Use the following keys to
    access the data -> Keys: 'day' - 'time' - 'info'
    
    create a final dictionary that will be used to store the information in the database as one row. 
    I will need to join the list items to create one string with a | seperating each item so I can 
    split the string when retrieving the data.
    """
    row = {}
    try:
        row['name'] = item0['name']
        row['address'] = item0['address']
        row['city'] = item0['city']
        row['state'] = item0['state']
    except Exception as e:
        print(e)
        row['name'] = 'name'
        row['address'] = 'address'
        row['city'] = 'city'
        row['state'] = 'state'

    # now add item1 to the row data
    row['day'] = ' | '.join(item1['day'])
    row['time'] = ' | '.join(item1['time'])
    row['info'] = ' | '.join(item1['info'])

    # now return the row data dictionary
    return row

# TODO: CREATE A FUNCTION THAT WILL WRITE EACH ITEM IN THE ROW_DATA LIST TO A CSV FILE
# SAVING THE ROW VALUES IN THEIR KEY'S COLUMN:
def convert_row_data_to_csv(row_data):
    with open('meeting_details.csv', 'w') as csvfile:
        for d in row_data:
            csvwriter = csv.DictWriter(csvfile, delimiter=',', field_names=list(d.keys()))
            csvwriter.writerow(d)
            
# TODO: GET THE LINKED LIST FROM THE CSV FILE IN meetings.csv
csv_filename = './meetings.csv'

# create func to get list of links:
def get_links(csv_filename):
    with open(csv_filename, 'r') as f:
        csv_reader = csv.reader(f)
        link_list = csv_parser(csv_reader, 'link')
    return link_list

# TODO: RETRIEVE THAT COMBINED DICT, AND APPEND TO AN EMPTY LIST CALLED ROW DATA

"""
functionality: 

1) Create a for loop, that will take each link in the linked list and it will, 
create a soup obj and return the contents:


"""

In [ ]:
def add_row_data_to_list(data: dict):
    # empty list to append individual data to
    dict_list = []

    # Now loop through the CSV file and append to the data list:
    for k, v in data.items():
        row_data = {k: v}
        dict_list.append(row_data)
    return dict_list
